In [17]:
import pandas as pd

In [18]:
csv_file = "dataset/bigcodebench_results.csv"
df = pd.read_csv(csv_file)

In [19]:
df.head(1)

,model,link,moe,size,act_param,type,complete,instruct
0,Magicoder-S-DS-6.7B,https://huggingface.co/ise-uiuc/Magicoder-S-DS...,False,6.7,6.7,🔶,47.6,36.2


In [20]:
new_rows = [
    {
        "model": "gemma-3-4b-it",
        "link": "https://huggingface.co/google/gemma-3-4b-it",
        "moe": "No",
        "size": "4B",
        "act_param": 4.0,
        "type": "decoder",
        "complete": 37.9,
        "instruct": None
    },
    {
        "model": "Hermes-2-Theta-Llama-3-8B",
        "link": "https://huggingface.co/NousResearch/Hermes-2-Theta-Llama-3-8B",
        "moe": "No",
        "size": "8B",
        "act_param": 8.0,
        "type": "decoder",
        "complete": 36.4,
        "instruct": None
    },
    {
        "model": "CodeLlama-7b-Instruct-hf",
        "link": "https://huggingface.co/meta-llama/CodeLlama-7b-Instruct-hf",
        "moe": "No",
        "size": "7B",
        "act_param": 7.0,
        "type": "decoder",
        "complete": 25.0,
        "instruct": None
    },
    {
        "model": "Phi-4-multimodal-instruct",
        "link": "https://huggingface.co/microsoft/Phi-4-multimodal-instruct",
        "moe": "No",
        "size": "5.57B",
        "act_param": 5.57,
        "type": "decoder",
        "complete": 46.5,
        "instruct": None
    },
]

In [21]:
# 3. Convert new rows to a DataFrame
df_new = pd.DataFrame(new_rows)

# 4. Append the new rows
df_combined = pd.concat([df, df_new], ignore_index=True)

df_combined.head()

C:\Users\slim\AppData\Local\Temp\ipykernel_28120\2290097208.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df, df_new], ignore_index=True)


,model,link,moe,size,act_param,type,complete,instruct
0,Magicoder-S-DS-6.7B,https://huggingface.co/ise-uiuc/Magicoder-S-DS...,False,6.7,6.7,🔶,47.6,36.2
1,StarCoder2-15B-Instruct-v0.1,https://huggingface.co/bigcode/starcoder2-15b-...,False,15.0,15.0,🔶,45.1,37.6
2,StarCoder2-3B,https://huggingface.co/bigcode/starcoder2-3b,False,3.0,3.0,🟢,21.4,NaN
3,StarCoder2-7B,https://huggingface.co/bigcode/starcoder2-7b,False,7.0,7.0,🟢,27.7,NaN
4,StarCoder2-15B,https://huggingface.co/bigcode/starcoder2-15b,False,15.0,15.0,🟢,38.4,NaN


In [22]:
# 5. Save back to CSV
df_combined.to_csv(csv_file, index=False)

In [1]:
from collections import Counter

def calculate_word_frequencies(conll_file_path):
    word_freq = Counter()

    with open(conll_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith("#"):
                continue

            parts = line.split("\t")
            if len(parts) >= 1:
                word = parts[0].lower()  # case-insensitive
                word_freq[word] += 1

    return word_freq

# Example usage
file_path = "Spanglish_train.conll"  # update if needed
word_frequencies = calculate_word_frequencies(file_path)

# Print top 10 most frequent words
for word, count in word_frequencies.most_common(10):
    print(f"{word}: {count}")


meta: 12005
.: 4503
de: 3797
:: 3057
a: 2985
,: 2897
!: 2745
que: 2663
el: 2259
y: 2254


In [10]:
from collections import Counter
import pandas as pd

def extract_lang1_lang2_words(conll_file_path):
    lang_words = []

    with open(conll_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("meta"):
                continue  # Skip meta lines or empty lines

            parts = line.split("\t")
            if len(parts) >= 2:
                word, lang = parts[0], parts[1]
                if lang in ("lang1", "lang2"):
                    lang_words.append(word.lower())

    return lang_words

# Step 1: Extract words
file_path = "Spanglish_train.conll"  # Update with your file path
filtered_words = extract_lang1_lang2_words(file_path)

# Step 2: Count frequencies
word_freq = Counter(filtered_words)

In [11]:
# Print top 10 most frequent words
for word, count in word_freq.most_common(10):
    print(f"{word}: {count}")

de: 3795
a: 2967
que: 2663
el: 2254
y: 2254
me: 2167
en: 2025
la: 2014
no: 1942
i: 1565


In [12]:
import pyarrow as pa
import pyarrow.parquet as pq

In [13]:
# Function to save the word frequencies to a Parquet file
def save_to_parquet(word_freq, output_file):
    # Convert the Counter to a DataFrame
    df = pd.DataFrame(word_freq.items(), columns=['Word', 'Count'])
    # Convert the DataFrame to a PyArrow Table
    table = pa.Table.from_pandas(df)
    # Write the table to a Parquet file
    pq.write_table(table, output_file)

save_to_parquet(word_freq, "word_frequency_spa.parquet")

In [7]:
import pyarrow.parquet as pq

def load_from_parquet(input_file):
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(input_file)
    # Convert the table to a pandas DataFrame
    df = table.to_pandas()
    # Convert the DataFrame to a dictionary for efficient lookup
    return pd.Series(df['Count'].values, index=df['Word'].str.lower()).to_dict()

input_file = "word_frequency.parquet"

# Load the word frequencies from the Parquet file
word_freq_dict = load_from_parquet(input_file)